In [7]:
import torch
from sentence_transformers import SentenceTransformer
from converter import sentence_transformers_onnx

In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [9]:
# load the model
model = SentenceTransformer.load('./results/domain_adaptation_model/')

In [10]:
onnx_model = sentence_transformers_onnx(
    model,
    path = "triton/model_repository/domain_adaptation_model/1/model"
)
onnx_model

In [19]:
import numpy as np
from torchvision import transforms
from PIL import Image
import tritonclient.http as httpclient
from tritonclient.utils import triton_to_np_dtype

# Setting up client
client = httpclient.InferenceServerClient(url="localhost:8000")

inputs = httpclient.InferInput("input__0", (3,), datatype="FP32")
inputs.set_data_from_numpy(np.array([1,2,3], dtype='float32'), binary_data=True)

outputs = httpclient.InferRequestedOutput("output__0", binary_data=True, class_count=1000)

# Querying the server
results = client.infer(model_name="./results/domain_adaptation_model/", inputs=[inputs], outputs=[outputs])
inference_output = results.as_numpy('start')
print(inference_output[:5])

InferenceServerException: [404] Not Found